Our solution utilizes some tricks that were public and some that we kept secret. Please don't use these in the future. You guys are making it too hard. 

In [1]:
seed_nb=16
import numpy as np 
np.random.seed(seed_nb)
import tensorflow as tf
tf.set_random_seed(seed_nb)

%matplotlib inline
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
#os.environ['CUDA_VISIBLE_DEVICES']='0'
import string
import re
#from sklearn.model_selection import train_test_split

#from sklearn.utils.class_weight import compute_class_weight
from gensim.models import KeyedVectors
import matplotlib.pyplot as plt
#from sklearn.metrics import f1_score as off1
#from sklearn.metrics import log_loss
import gc
from tqdm import tqdm
from tqdm import tqdm_notebook
import time
import h5py
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
tqdm.pandas()

Using TensorFlow backend.


In [2]:
INPUT_PATH = '../input/'
print(os.listdir(INPUT_PATH))
debug = False
do_submit = True
simulate_test = False
opt = "val_loss"

['embeddings', 'train.csv', 'sample_submission.csv', 'test.csv']


One thing we did along the way was test out how our model would perform with a much larger testing set. We did this simply by loading in the test set multiple times and concatenating.

In [3]:
train_df = pd.read_csv(INPUT_PATH + 'train.csv')
# if submit use original test set else split off from train
if simulate_test:
    train_df, test_df = train_test_split(train_df, test_size = 0.1, random_state = 28)
else:
    test_df = pd.read_csv(INPUT_PATH + 'test.csv')
#     for i in range(6):
#         test_df = pd.concat([pd.read_csv(INPUT_PATH + 'test.csv'), test_df], axis = 0)
print(train_df.shape)
print(test_df.shape)

(1306122, 3)
(375806, 2)


As most others have already shown in their strong placing kernels, preprocessing was very important. I think we probably could have done better here. It looks like many of the lemmatizing and tokenizing tricks we wrote off were actually useful. You can see how we went through a progression of text preprocessing based off of what is commented out. For a long time we were using the preprocessing that was publicly available in @dieters kernel https://www.kaggle.com/christofhenkel/how-to-preprocessing-when-using-embeddings. Eventually we settled on the below preprocessing by systematically testing each option  and combination of options and seeing how they performed. 

In [4]:
import string
from nltk.tokenize.treebank import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()
# constants

# from https://stackoverflow.com/questions/2013451/test-if-string-contains-only-letters-a-z-é-ü-ö-ê-å-ø-etc
latin_similar = "’'‘ÆÐƎƏƐƔĲŊŒẞÞǷȜæðǝəɛɣĳŋœĸſßþƿȝĄƁÇĐƊĘĦĮƘŁØƠŞȘŢȚŦŲƯY̨Ƴąɓçđɗęħįƙłøơşșţțŧųưy̨ƴÁÀÂÄǍĂĀÃÅǺĄÆǼǢƁĆĊĈČÇĎḌĐƊÐÉÈĖÊËĚĔĒĘẸƎƏƐĠĜǦĞĢƔáàâäǎăāãåǻąæǽǣɓćċĉčçďḍđɗðéèėêëěĕēęẹǝəɛġĝǧğģɣĤḤĦIÍÌİÎÏǏĬĪĨĮỊĲĴĶƘĹĻŁĽĿʼNŃN̈ŇÑŅŊÓÒÔÖǑŎŌÕŐỌØǾƠŒĥḥħıíìiîïǐĭīĩįịĳĵķƙĸĺļłľŀŉńn̈ňñņŋóòôöǒŏōõőọøǿơœŔŘŖŚŜŠŞȘṢẞŤŢṬŦÞÚÙÛÜǓŬŪŨŰŮŲỤƯẂẀŴẄǷÝỲŶŸȲỸƳŹŻŽẒŕřŗſśŝšşșṣßťţṭŧþúùûüǔŭūũűůųụưẃẁŵẅƿýỳŷÿȳỹƴźżžẓ"
white_list = string.ascii_letters + string.digits + latin_similar + ' '
# from checking symbols_iv = {char:char_vocab[char] for char in symbols if char in embeddings_index}
symbols_iv = """?,./-()"$=…*&+′[ɾ̃]%:^\xa0\\{}–“”;!<`®ạ°#²|~√_α→>—£，。´×@π÷？ʿ€の↑∞ʻ℅в•−а年！∈∩⊆§℃θ±≤͡⁴™си≠∂³ி½△¿¼∆≥⇒¬∨∫▾Ω＾γµº♭ー̂ɔ∑εντσ日Γ∪φβ¹∘¨″⅓ɑː✅✓（）∠«»்ுλ∧∀،＝ɨʋδɒ¸☹μΔʃɸηΣ₅₆◦·ВΦ☺❤♨✌≡ʌʊா≈⁰‛：ﬁ„¾ρ⟨⟩˂⅔≅－＞¢⁸ʒは⬇♀؟¡⋅ɪ₁₂ɤ◌ʱ、▒ْ；☉＄∴✏ωɹ̅।ـ☝♏̉̄♡₄∼́̀⁶⁵¦¶ƒˆ‰©¥∅・ﾟ⊥ª†ℕ│ɡ∝♣／☁✔❓∗➡ℝ位⎛⎝¯⎞⎠↓ɐ∇⋯˚⁻ˈ₃⊂˜̸̵̶̷̴̡̲̳̱̪̗̣̖̎̿͂̓̑̐̌̾̊̕\x92"""

def get_chars(train_X):
    char_vocab = {}
    for sent in tqdm_notebook(train_X):
        for char in sent:
            try:
                char_vocab[char] += 1
            except KeyError:
                char_vocab[char] = 1
    return char_vocab

def split_off_symbols_iv(x):
    for punct in symbols_iv:
        x = x.replace(punct, f' {punct} ')
    return x

def preprocess(text):
    
    text = split_off_symbols_iv(text) #increase score
    #text = delete_oov_symbols(text,translate_map)
    #text = ' '.join(text.split())
    #text = clean_apostrophes(text)
    #text = clean_contractions(text, contraction_mapping)
    #text = split_off_s(text)
    #text = replace_acronyms(text,acronyms)
    ##text = fix_mispells(text,mispells) decrease score
    #text = fix_apo(text)
    return text

In [5]:
char_vocab = get_chars(train_df["question_text"].values)
symbols = {char:char_vocab[char] for char in char_vocab if not char in white_list}

In [6]:
train_df["question_text"] = train_df["question_text"].progress_apply(lambda x: preprocess(x))
train_df["question_text"] = train_df["question_text"].progress_apply(lambda x: tokenizer.tokenize(x, return_str=True))
test_df["question_text"] = test_df["question_text"].progress_apply(lambda x: preprocess(x))
test_df["question_text"] = test_df["question_text"].progress_apply(lambda x: tokenizer.tokenize(x, return_str=True))

100%|██████████| 375806/375806 [00:38<00:00, 9769.23it/s]


This is all roughly standard stuff. Only thing I think we could have improved here was a longer maxlen and larger vocabulary size, but we were having issues early on with running out of memory and killing kernels. We never really revisited this, but it seems everyone else was able to fit in many more words and multiple embeddings at a time. 

In [7]:
# some config values 
embed_size = 300 # how big is each word vector
max_features = 100000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 50 # max number of words in a question to use

train_X = train_df["question_text"].fillna("_na_").values
test_X = test_df["question_text"].fillna("_na_").values
## Tokenize the sentences
tokenizer = Tokenizer(num_words=max_features, filters = '', lower = False)
tokenizer.fit_on_texts(list(train_X) + list(test_X) )
train_X = tokenizer.texts_to_sequences(train_X)
test_X = tokenizer.texts_to_sequences(test_X)
## Pad the sentences 
train_X = pad_sequences(train_X, maxlen=maxlen)
test_X = pad_sequences(test_X, maxlen=maxlen)
word_index = tokenizer.word_index

In [8]:
del tokenizer
gc.collect()

0

Using the gensim function to load in the embeddings ended up being much more time efficient than most of the things available in the public kernels. I am sure a few other people used this same function but I did not see it widespread. 

In [9]:
from gensim import utils

def load_word2vec(fname, encoding='utf8', unicode_errors='strict',datatype=np.float32, max_vocab=3000000, word_index=None):
    #emb_mean,emb_std = -0.0051106834, 0.18445626
    #embedding_matrix = np.random.normal(emb_mean, emb_std, (max_features, 300))
    embedding_index = {}
    with utils.smart_open(fname) as fin:
        header = utils.to_unicode(fin.readline(), encoding=encoding)
        vocab_size, vector_size = (int(x) for x in header.split())
        binary_len = np.dtype(datatype).itemsize * vector_size
        
        for _ in tqdm(range(min(vocab_size,max_vocab))):
            # mixed text and binary: read text first, then binary
            word = []
            while True:
                ch = fin.read(1)
                if ch == b' ':
                    break
                if ch == b'':
                    raise EOFError("unexpected end of input")
                if ch != b'\n':
                    word.append(ch)
            word = utils.to_unicode(b''.join(word), encoding=encoding, errors=unicode_errors)
            weights = np.fromstring(fin.read(binary_len), dtype=datatype).astype(np.float16)
            embedding_index[word] = weights
    return embedding_index

Our functions for loading our embeddings

In [10]:
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')

def load_news(embed_dir = '../input/GoogleNews-vectors-negative300.bin'):
    embeddings_index = KeyedVectors.load_word2vec_format(embed_dir, binary=True)
    emb_ind = {}
    for i, vec in tqdm(enumerate(embeddings_index.wv.vectors)):
        emb_ind[embeddings_index.wv.index2word[i]] = vec
    del embeddings_index
    gc.collect()
    return emb_ind
        
def load_glove(word_index,embed_dir = "../input/embeddings/glove.840B.300d/glove.840B.300d.txt"):
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in tqdm_notebook(open(embed_dir, encoding = "utf8")) if o.split(" ")[0] in word_index) 
    return embeddings_index

def load_para(embed_dir ='../input/embeddings/paragram_300_sl999/paragram_300_sl999.txt'):
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in tqdm_notebook(open(embed_dir, encoding="utf8", errors='ignore')) if len(o)>100)
    return embeddings_index

Our standard embedding matrix builder. We will revisit this in a second because we heavily altered things after the fact.

In [11]:
def build_embedding_matrix(word_index,embeddings_index, max_features, maxlen, lower = False):
    #all_embs = np.stack(embeddings_index.values())
    #emb_mean,emb_std = all_embs.mean(), all_embs.std()

    #embedding_matrix = np.random.normal(emb_mean, emb_std, (max_features, 300), dty)
    embedding_matrix = np.zeros((max_features, 300))
    for word, i in word_index.items():
        if lower:
            word = word.lower()
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: 
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

We only ended up using two of the below functions, add_lower_and_upper and add_symbols. What these do are add things to our embeddings which were missing. So if a word didnt show up in our embedding matrix, but we found that the lower case or uppercase was available in the pretrained embeddings then we would add that in where the blanks were. 

In [12]:
def add_lower_and_upper(embedding, vocab):
    count_l = 0
    count_u = 0
    for word in tqdm(vocab):
        if word not in embedding and word.lower() in embedding:  
            embedding[word] = embedding[word.lower()]
            count_l += 1
        if word not in embedding and word.title() in embedding:
            embedding[word] = embedding[word.title()]
            count_u += 1
    print(f"Added {count_l} lower word embeddings")
    print(f"Added {count_u} upper word embeddings")

def clean_numbers(x):

    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x

def add_numbers(embedding, word_index):
    count_d = 0
    count_s = 0
    for word in tqdm(word_index):
        if word not in embedding:
            cln = clean_numbers(word)
            if cln in embedding:  
                embedding[word] = embedding[cln]
                count_d += 1
    print(f"Added {count_d} number embeddings")
    
def add_symbols(embedding, word_index):
    
    quotes=["``","''"]
    symbs_iv = [char for char in symbols if char in embedding]
    mean_symbol = np.zeros((300,))
    for s in symbs_iv:
        mean_symbol += embedding[s]
    
    mean_symbol /= len(symbs_iv)

    count_s = 0
    for word in tqdm(word_index):
        if word not in embedding:
            if len(word) == 1:
                embedding[word] = mean_symbol
                count_s += 1
            if word in quotes:
                embedding[word] = mean_symbol
                count_s += 1
    print(f"Added {count_s} symbol embeddings")
    
def build_glove_matrix():
    embeddings_index = load_glove(word_index=word_index,embed_dir = INPUT_PATH + "embeddings/glove.840B.300d/glove.840B.300d.txt")
    add_lower_and_upper(embeddings_index, word_index)
    add_symbols(embeddings_index, word_index)
    embedding_matrix1 = build_embedding_matrix(word_index,embeddings_index, max_features, maxlen, lower = False)
    return embedding_matrix1

In [13]:
embedding_matrix1 = build_glove_matrix()

 31%|███       | 85738/280439 [00:00<00:00, 857376.73it/s]

 41%|████      | 113646/280439 [00:00<00:00, 1136458.37it/s]

Added 1310 lower word embeddings
Added 1922 upper word embeddings


100%|██████████| 280439/280439 [00:00<00:00, 1121553.85it/s]


Added 351 symbol embeddings


In [14]:
gc.collect()

0

In [15]:
train_y = train_df["target"].values
if simulate_test: 
    test_y = test_df["target"].values
else:
    test_y = None

In [16]:
from keras.models import Model, load_model
from keras.layers import Input, Dense, Embedding, concatenate, CuDNNGRU, CuDNNLSTM, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D, SpatialDropout1D, Flatten, Lambda, Permute, Reshape, merge, Dropout, Conv2D, MaxPool2D, Concatenate, Conv1D, MaxPool1D, add, MaxPooling1D
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from keras import backend as K
from keras.losses import binary_crossentropy

from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.initializers import glorot_uniform, he_uniform, he_normal
from keras.optimizers import Adam
from keras.layers import LeakyReLU, multiply
from keras.layers import Reshape, Permute, multiply

Here is where our first major trick occurred. Full credit for Dieter for noticing this. What we did here was actually really clever to save time. After looking at the histograms of our predictions Dieter noticed that almost 70 percent of our predictions were under .01 meaning that the model was very confident that they were sincere questions. We also measured the accuracy of thes predictions and found that it was nearly perfect. Our models were able to find these easy ones within a few epochs. We exploited this by training one model in order to filter out the bottom 70 percent of the data and then retraining all of our models on the remaining 30%. This allowed us to train many more models in a quick span and more completely. After we train this model we simply throw it away so as not to leak any results into our validation set. All models after this one are trained from scratch only on the subset of data 

In [17]:
def build_gru_model(embedding_matrix, name = "gru"):
    
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features,300,weights=[embedding_matrix],input_length=maxlen,trainable=False)(inp)
    x = SpatialDropout1D(.4,seed=seed_nb)(x)
    x = CuDNNGRU(128, return_sequences=True,kernel_initializer=glorot_uniform(seed=seed_nb))(x)
    x = CuDNNGRU(64, return_sequences=True,kernel_initializer=glorot_uniform(seed=seed_nb))(x)
    last = Flatten()(Lambda(lambda x: x[:,-1:,:]) (x))
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool, last])
    #x = Dropout(.3,seed=seed_nb)(x)
    x = Dense(32, activation = 'relu',kernel_initializer=he_uniform(seed=seed_nb))(conc)
    outp = Dense(1, activation="sigmoid",kernel_initializer=he_uniform(seed=seed_nb))(x)
    
    model = Model(inputs=[inp], outputs=outp, name = name)
    return model

In [18]:
K.clear_session()
batch_size = 2048
model = build_gru_model(embedding_matrix1, name = 'gru_stage0')
model.compile(loss=binary_crossentropy,optimizer=Adam())
model.fit(train_X, train_y, batch_size=batch_size, epochs=3,verbose=True)

Epoch 1/3
1306122/1306122 [==============================] - 85s 65us/step - loss: 0.1314
Epoch 2/3
1306122/1306122 [==============================] - 84s 64us/step - loss: 0.1109
Epoch 3/3
1306122/1306122 [==============================] - 84s 64us/step - loss: 0.1059


In [19]:
from sklearn.metrics import confusion_matrix, accuracy_score, log_loss, f1_score


filter_pred = model.predict(train_X, batch_size = batch_size, verbose =True)[:,0]
thresh = np.percentile(filter_pred,70.)
print(thresh)
keep_index = (filter_pred > thresh)
#trash_index = np.logical_not(keep_index)
#trashed_pred = filter_pred[trash_index]
#trashed_train = train_X[trash_index]
#trashed_y = train_y[trash_index]

#print('trashing data with these metrics:')
#print('confusion matrix:')
#print(confusion_matrix(trashed_pred>0.5,trashed_y))
#print('acc:')
#print(accuracy_score(trashed_pred>0.5,trashed_y))

1306122/1306122 [==============================] - 28s 21us/step
0.009684420283883804


In [20]:
#k = np.where(keep_index)
#t = np.where(np.logical_not(keep_index))
#train_X_old = train_X.copy()
#train_y_old = train_y.copy()
train_X = train_X[np.where(keep_index)]
train_y = train_y[np.where(keep_index)]

In [21]:
train_X.shape

(391837, 50)

Now you can see that our training set is much smaller and we only need to focus on the difficult problems. It is a waste of time in order to continue training on training samples the model is already confident and accurate on. These examples are trivial for one reason or another. We found that dropping these samples does not negatively affect our accuracy at all and our models trained on the subset can still easily classify those that were dropped and never trained on. 

In [22]:
del filter_pred, model
gc.collect()

40

After filtering out these samples we used 5 models. These were found by training an array of models with different embeddings and configurations and then seeing how they ensembled together. Our offline stacker eventually found that the best ensemble of models was:
* DPCNN with reversing and glove embeddings
* A bidirectional gru into an lstm with the glove embeddings. (this was very similar to what we used for the toxic comment challenge and was our strongest individual model here as well)
* a parrallel lstm and gru model w/glove embeddings
* parts of speech bidirectional lstm and gru model w/paragram embeddings
* parts of speech parallel lstm and gru model w/news embeddings

I will talk about the POS models more in a second

One thing I wish we had explored a bit more was stacked embeddings like many used. We simply were running into too many issues early on with killing kernels so I wrote it off as a viable option. In the toxic comment challenge we heavily utilized this with even stacking 3 embeddings at once and also encoding our own information in the embeddings like if the word was all_caps and other various features. We simply didnt have the throughput in order to use all of these techniques at once. 

In [23]:
class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

def squash(x, axis=-1):
    # s_squared_norm is really small
    # s_squared_norm = K.sum(K.square(x), axis, keepdims=True) + K.epsilon()
    # scale = K.sqrt(s_squared_norm)/ (0.5 + s_squared_norm)
    # return scale * x
    s_squared_norm = K.sum(K.square(x), axis, keepdims=True)
    scale = K.sqrt(s_squared_norm + K.epsilon())
    return x / scale

# A Capsule Implement with Pure Keras
class Capsule(Layer):
    def __init__(self, num_capsule, dim_capsule, routings=3, kernel_size=(9, 1), share_weights=True,
                 activation='default', **kwargs):
        super(Capsule, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings
        self.kernel_size = kernel_size
        self.share_weights = share_weights
        if activation == 'default':
            self.activation = squash
        else:
            self.activation = Activation(activation)

    def build(self, input_shape):
        super(Capsule, self).build(input_shape)
        input_dim_capsule = input_shape[-1]
        if self.share_weights:
            self.W = self.add_weight(name='capsule_kernel',
                                     shape=(1, input_dim_capsule,
                                            self.num_capsule * self.dim_capsule),
                                     # shape=self.kernel_size,
                                     initializer='glorot_uniform',
                                     trainable=True)
        else:
            input_num_capsule = input_shape[-2]
            self.W = self.add_weight(name='capsule_kernel',
                                     shape=(input_num_capsule,
                                            input_dim_capsule,
                                            self.num_capsule * self.dim_capsule),
                                     initializer='glorot_uniform',
                                     trainable=True)

    def call(self, u_vecs):
        if self.share_weights:
            u_hat_vecs = K.conv1d(u_vecs, self.W)
        else:
            u_hat_vecs = K.local_conv1d(u_vecs, self.W, [1], [1])

        batch_size = K.shape(u_vecs)[0]
        input_num_capsule = K.shape(u_vecs)[1]
        u_hat_vecs = K.reshape(u_hat_vecs, (batch_size, input_num_capsule,
                                            self.num_capsule, self.dim_capsule))
        u_hat_vecs = K.permute_dimensions(u_hat_vecs, (0, 2, 1, 3))
        # final u_hat_vecs.shape = [None, num_capsule, input_num_capsule, dim_capsule]

        b = K.zeros_like(u_hat_vecs[:, :, :, 0])  # shape = [None, num_capsule, input_num_capsule]
        for i in range(self.routings):
            b = K.permute_dimensions(b, (0, 2, 1))  # shape = [None, input_num_capsule, num_capsule]
            c = K.softmax(b)
            c = K.permute_dimensions(c, (0, 2, 1))
            b = K.permute_dimensions(b, (0, 2, 1))
            outputs = self.activation(tf.keras.backend.batch_dot(c, u_hat_vecs, [2, 2]))
            if i < self.routings - 1:
                b = tf.keras.backend.batch_dot(outputs, u_hat_vecs, [2, 3])

        return outputs

    def compute_output_shape(self, input_shape):
        return (None, self.num_capsule, self.dim_capsule)

In [24]:
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

One major improvement we were able to add to all of our models because of this time savings was much higher spatial dropout and also a secondary training phase where we trained only the embedding layer. 

Spatial Dropout we found to be exceptionally effective to prevent overfitting in the toxic comment challenge. What this does is mask out dimensions of the embeddings. This is well described a couple places in the forums https://www.kaggle.com/c/quora-insincere-questions-classification/discussion/79911. The key is that your model is only seeing subsets of the dimensions of the embeddings at a time so it does not fit to all of them at once. Some people were doing this incorrectly in the pytorch variant where Dropout2D was dropping out entire words rather than just dimensions of the embeddings. We did some experiments of our own with PyTorch and confirmed this was not correct so @MihaS tried to correct this by manually creating a new dropout layer for us. We did not end up spending much more time with PyTorch though because the majority of us were proficient with Keras. 

The second portion, finetuning the embeddings is very crucial. Many people I saw used either unfrozen embeddings the entire time or left them frozen the entire training process. In my opinion this is not the correct way to approach it. If we look at how we do transfer learning with imagenet models and the like what people typically do is freeze the network except for the last layer and train that and then tune the rest of the network after that last layer is at least somewhat trained. The reason behind doing this is if you train the entire network and your last layer is randomly initialized then it will essentially just blow out whatever was there because the gradient will be so noisy. If you do that you are using them as initialization points rather than to actually transfer knowledge to a new problem. The same is true with word embeddings. We train the rest of the network around the embeddings and then once that is adequately trained we can do some tweaking of the embeddings. In this way we are able to maintain all of the juicy information captured in these embeddings and also impart some of the information from this new dataset

In [25]:
def train_model(model,train_x, y_tra,epochs,batch_size):
    
    
    #complete_hist = {'model_name':model.name}
    #complete_hist['epochs'] = [0,0,0]
    #tic = time.time()
    model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['acc', f1])

    hist = model.fit(train_x, y_tra, batch_size=batch_size, epochs=epochs[1],verbose=True)

    for layer in model.layers:
        layer.trainable = False
    model.layers[1].trainable = True
    
    model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['acc', f1])
    hist = model.fit(train_x, y_tra, batch_size=batch_size, epochs=epochs[2],verbose=True)
    
    return hist

In [26]:
def build_dpcnn_model(embedding_matrix, num_block = 5, k = 3, units = 64,name = 'dpcnn'):
    pad = 'same'
    inp = Input(shape = (None, ))
    embedding_layer1 = Embedding(max_features, embed_size, weights=[embedding_matrix], embeddings_initializer = he_uniform(seed=seed_nb), trainable = False)(inp)
    embedding_layer1 = SpatialDropout1D(0.3)(embedding_layer1)
    emb_short_cut = Conv1D(units, kernel_size = 1, padding = pad, activation = 'relu')(embedding_layer1)
    emb_short_cut = Conv1D(units, kernel_size = 1, padding = pad, activation = 'relu')(emb_short_cut)
    # Main block
    for b in range(1, num_block + 1):
        if b == 1:
            block = embedding_layer1
            short_cut = emb_short_cut
        else:
            block = block
            short_cut = block
        block = Conv1D(units, kernel_size = k, padding = pad, activation = 'relu')(block)
        block = Conv1D(units, kernel_size = k, padding = pad, activation = 'relu')(block)
        block = add([short_cut, block])
        block = MaxPooling1D(pool_size = 3, strides = 2, padding = pad)(block)
    # Final block
    short_cut = block
    block = Conv1D(units, kernel_size = k, padding = pad, activation = 'relu')(block)
    block = Conv1D(units, kernel_size = k, padding = pad, activation = 'relu')(block)
    block = add([short_cut, block])
    max_pool = GlobalMaxPooling1D()(block)
    avg_pool = GlobalAveragePooling1D()(block)
    last = (Lambda(lambda x: x[:,-1,:]) (block))
    block = concatenate([max_pool, avg_pool, last])
    #reverse
    rev_embedding_layer = Lambda(lambda x: K.reverse(x,axes=-1))(embedding_layer1)
    rev_emb_short_cut = Conv1D(units, kernel_size = 1, padding = pad, activation = 'relu')(rev_embedding_layer)
    rev_emb_short_cut = Conv1D(units, kernel_size = 1, padding = pad, activation = 'relu')(rev_emb_short_cut)
    # Main block
    for b in range(1, num_block + 1):
        if b == 1:
            rev_block = rev_embedding_layer
            rev_short_cut = rev_emb_short_cut
        else:
            rev_block = rev_block
            rev_short_cut = rev_block
        rev_block = Conv1D(units, kernel_size = k, padding = pad, activation = 'relu')(rev_block)
        rev_block = Conv1D(units, kernel_size = k, padding = pad, activation = 'relu')(rev_block)
        rev_block = add([rev_short_cut, rev_block])
        rev_block = MaxPooling1D(pool_size = 3, strides = 2, padding = pad)(rev_block)
    # Final block
    rev_short_cut = rev_block
    rev_block = Conv1D(units, kernel_size = k, padding = pad, activation = 'relu')(rev_block)
    rev_block = Conv1D(units, kernel_size = k, padding = pad, activation = 'relu')(rev_block)
    rev_block = add([rev_short_cut, rev_block])
    rev_max_pool = GlobalMaxPooling1D()(rev_block)
    rev_avg_pool = GlobalAveragePooling1D()(rev_block)
    rev_last = (Lambda(lambda x: x[:,-1,:]) (rev_block))
    rev_block = concatenate([rev_max_pool, rev_avg_pool, rev_last])
    block = concatenate([rev_block, block])
    # output block
    out_put = Dense(64, activation = 'relu')(block)
    outp = Dense(1, activation="sigmoid")(out_put)
    model = Model(inputs=inp, outputs=outp, name=name)
    return model

In [27]:
n_models = 5
preds_test_all = np.zeros((test_X.shape[0],n_models))

In [28]:
K.clear_session()
model_name = 'dpcnn'
batch_size = 1000
epochs = [0,12,3] # checked locally

model = build_dpcnn_model(embedding_matrix1)
dpcnn_hist = train_model(model,train_X, train_y, epochs,batch_size)
preds_test_all[:,0] = model.predict(test_X, batch_size=batch_size,verbose = True)[:,0]

Epoch 1/12
391837/391837 [==============================] - 55s 141us/step - loss: 0.3880 - acc: 0.8391 - f1: 0.4805
Epoch 2/12
391837/391837 [==============================] - 51s 131us/step - loss: 0.3287 - acc: 0.8613 - f1: 0.6117
Epoch 3/12
391837/391837 [==============================] - 51s 131us/step - loss: 0.3147 - acc: 0.8663 - f1: 0.6334
Epoch 4/12
391837/391837 [==============================] - 51s 131us/step - loss: 0.3057 - acc: 0.8701 - f1: 0.6470
Epoch 5/12
391837/391837 [==============================] - 51s 131us/step - loss: 0.2983 - acc: 0.8732 - f1: 0.6589
Epoch 6/12
391837/391837 [==============================] - 51s 131us/step - loss: 0.2926 - acc: 0.8758 - f1: 0.6681
Epoch 7/12
391837/391837 [==============================] - 51s 131us/step - loss: 0.2874 - acc: 0.8777 - f1: 0.6748
Epoch 8/12
391837/391837 [==============================] - 51s 130us/step - loss: 0.2831 - acc: 0.8796 - f1: 0.6810
Epoch 9/12
391837/391837 [==============================] - 51s 

In [29]:

#ens_pred1 = model.predict(X_ens, batch_size=batch_size,verbose = True)
#os.remove("dpcnn.h5")

In [30]:
del model, dpcnn_hist
gc.collect()

237

In [31]:
def build_gru_lstm_model(embedding_matrix, name = ""):
    
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
    x = SpatialDropout1D(.4, seed = seed_nb)(x)
    x = Bidirectional(CuDNNGRU(128, return_sequences=True,kernel_initializer=glorot_uniform(seed=seed_nb)))(x)
    x = Dropout(.2, seed = seed_nb)(x)
    x = Bidirectional(CuDNNLSTM(64, return_sequences=True,kernel_initializer=glorot_uniform(seed=seed_nb)))(x)

    last = Flatten()(Lambda(lambda x: x[:,-1:,:]) (x))
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool,max_pool,last])
    x = Dropout(.3, seed = seed_nb)(x)
    outp = Dense(1, activation="sigmoid",kernel_initializer=he_uniform(seed=seed_nb))(conc)
    
    model = Model(inputs=[inp], outputs=outp, name = name)
    return model

In [32]:
K.clear_session()
model_name = 'gru_lstm'
batch_size = 2048
epochs = [0,12,3] # checked locally

model = build_gru_lstm_model(embedding_matrix1)
lstm_gru_hist = train_model(model,train_X, train_y, epochs,batch_size)
preds_test_all[:,1] = model.predict(test_X, batch_size=batch_size,verbose = True)[:,0]
del model, lstm_gru_hist
gc.collect()

Epoch 1/12
391837/391837 [==============================] - 65s 166us/step - loss: 0.4008 - acc: 0.8284 - f1: 0.4102
Epoch 2/12
391837/391837 [==============================] - 64s 163us/step - loss: 0.3472 - acc: 0.8528 - f1: 0.5732
Epoch 3/12
391837/391837 [==============================] - 64s 163us/step - loss: 0.3303 - acc: 0.8609 - f1: 0.6097
Epoch 4/12
391837/391837 [==============================] - 64s 163us/step - loss: 0.3202 - acc: 0.8652 - f1: 0.6277
Epoch 5/12
391837/391837 [==============================] - 64s 163us/step - loss: 0.3130 - acc: 0.8683 - f1: 0.6403
Epoch 6/12
391837/391837 [==============================] - 64s 163us/step - loss: 0.3075 - acc: 0.8701 - f1: 0.6466
Epoch 7/12
391837/391837 [==============================] - 64s 163us/step - loss: 0.3010 - acc: 0.8733 - f1: 0.6580
Epoch 8/12
391837/391837 [==============================] - 64s 163us/step - loss: 0.2975 - acc: 0.8745 - f1: 0.6627
Epoch 9/12
391837/391837 [==============================] - 64s 

314083

In [33]:
def build_parallel_lstm_gru(embedding_matrix, name = ""):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
    x = SpatialDropout1D(.4, seed = seed_nb)(x)
    x1 = CuDNNGRU(128, return_sequences=True,kernel_initializer=glorot_uniform(seed=seed_nb))(x)
    x2 = CuDNNLSTM(128, return_sequences=True,kernel_initializer=glorot_uniform(seed=seed_nb))(x)
    x = Concatenate()([x1,x2])
    x = Dropout(.2, seed = seed_nb)(x)
    x1 = CuDNNGRU(64, return_sequences=True,kernel_initializer=glorot_uniform(seed=seed_nb))(x)
    x2 = CuDNNLSTM(64, return_sequences=True,kernel_initializer=glorot_uniform(seed=seed_nb))(x)
    x = Concatenate()([x1,x2])
    last = Flatten()(Lambda(lambda x: x[:,-1:,:]) (x))
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool,  last])
    #x = Dropout(.3, seed = seed_nb)(x)
    outp = Dense(1, activation="sigmoid",kernel_initializer=he_uniform(seed=seed_nb))(conc)
    
    model = Model(inputs=[inp], outputs=outp, name = name)
    return model

In [34]:
K.clear_session()
model_name = 'parallel_lstm_gru2'
batch_size = 2048
epochs = [0,11,4] # checked locally

model = build_parallel_lstm_gru(embedding_matrix1)
parallel_lstm_gru2_hist = train_model(model,train_X, train_y, epochs,batch_size)
preds_test_all[:,2] = model.predict(test_X, batch_size=batch_size,verbose = True)[:,0]
del model, parallel_lstm_gru2_hist
gc.collect()

Epoch 1/11
391837/391837 [==============================] - 63s 160us/step - loss: 0.3954 - acc: 0.8312 - f1: 0.4348
Epoch 2/11
391837/391837 [==============================] - 62s 157us/step - loss: 0.3464 - acc: 0.8534 - f1: 0.5756
Epoch 3/11
391837/391837 [==============================] - 62s 157us/step - loss: 0.3308 - acc: 0.8606 - f1: 0.6067
Epoch 4/11
391837/391837 [==============================] - 62s 157us/step - loss: 0.3195 - acc: 0.8655 - f1: 0.6273
Epoch 5/11
391837/391837 [==============================] - 62s 157us/step - loss: 0.3126 - acc: 0.8684 - f1: 0.6388
Epoch 6/11
391837/391837 [==============================] - 62s 157us/step - loss: 0.3062 - acc: 0.8706 - f1: 0.6483
Epoch 7/11
391837/391837 [==============================] - 62s 157us/step - loss: 0.3010 - acc: 0.8731 - f1: 0.6570
Epoch 8/11
391837/391837 [==============================] - 62s 157us/step - loss: 0.2970 - acc: 0.8742 - f1: 0.6615
Epoch 9/11
391837/391837 [==============================] - 62s 

174

The below section is where we did POS tagging in order to disambiguate the words. I have a more detailed write up of this in my other public kernel https://www.kaggle.com/ryches/parts-of-speech-disambiguation-error-analysis. This technique on its own performs worse (I have some hypotheses of why this might be the case. It has to do with the way I am initializing them in the same spot. If I were to do something to break their symmetry I think it would likely perform even better) but when ensembled, since the model is learning based off of slightly more expressive embeddings, it actually performs significantly better. The offline stacker gave these much higher weights than initially expected. I used this same technique in toxic comment, but I was not able to extensively test how it worked and my method is much more elegant now. Last competition I simply sent the results to alex and he gave me the thumbs up saying it seemed to get weight in the stacker because we found out so late in the competition. This time I got to actually play with it and see it for myself. This took our 5 model cv from .702 to upper .705. when stacked. We were only able to do this by training on the subset. With a full training set we would have run out of time. Especially with the expanded test set. 

Our other submission was able to fit 8 models in one kernel, but it turns out it was worth dropping those models in place of the POS tagging. 

There are various difficulties with the POS tagging, specifically making the text actually line up with the parts of speech that are output. Keras tokenizer will typically break these so I had to take care with this.

In [35]:
train_X = train_df["question_text"].fillna("_na_").values
test_X = test_df["question_text"].fillna("_na_").values
train_X = train_X[np.where(keep_index)]
from spacy.tokens import Doc
from spacy.vocab import Vocab
import spacy
class WhitespaceTokenizer(object):
    def __init__(self, vocab):
        self.vocab = vocab
    def __call__(self, text):
        words = text.strip().split(' ')
        words = [word for word in words if word is not '']
        spaces = [True] * len(words)
        return Doc(self.vocab, words=words, spaces=spaces)
nlp = spacy.load('en', disable = ["parser", "ner", "textcat"])
nlp.tokenizer = WhitespaceTokenizer(nlp.vocab)
tokens = []
for doc in tqdm(nlp.pipe(train_X, n_threads = 8)):
    tokens.append(" ".join([n.text + "_"  + n.pos_ for n in doc]))
train_X = tokens
tokens = []
for doc in tqdm(nlp.pipe(test_X, n_threads = 8)):
    tokens.append(" ".join([n.text + "_"  + n.pos_ for n in doc]))
test_X = tokens
## Tokenize the sentences
tokenizer = Tokenizer(num_words=max_features, filters = '', lower = False)
tokenizer.fit_on_texts(list(train_X) + list(test_X) )
train_X = tokenizer.texts_to_sequences(train_X)
test_X = tokenizer.texts_to_sequences(test_X)
## Pad the sentences 
train_X = pad_sequences(train_X, maxlen=maxlen)
test_X = pad_sequences(test_X, maxlen=maxlen)
word_index = tokenizer.word_index

391837it [05:54, 1105.96it/s]
375806it [04:56, 1268.27it/s]


In [36]:
def build_embedding_matrix(word_index,embeddings_index, max_features, maxlen, lower = False):
    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()

#     embedding_matrix = np.random.normal(emb_mean, emb_std, (max_features, 300))
    embedding_matrix = np.zeros((max_features, 300))
    for word, i in word_index.items():
        if i >= max_features: continue
        word_part = word.split("_")[0]
        embedding_vector = embeddings_index.get(word_part)
        if embedding_vector is not None: 
            embedding_matrix[i] = embedding_vector
    return embedding_matrix
def add_lower_and_upper(embedding, vocab):
    count_l = 0
    count_u = 0
    for word in tqdm(vocab):
        word = word.split("_")[0]
        if word not in embedding and word.lower() in embedding:  
            embedding[word] = embedding[word.lower()]
            count_l += 1
        if word not in embedding and word.title() in embedding:
            embedding[word] = embedding[word.title()]
            count_u += 1
    print(f"Added {count_l} lower word embeddings")
    print(f"Added {count_u} upper word embeddings")
embeddings_index2 = load_para(embed_dir = INPUT_PATH + 'embeddings/paragram_300_sl999/paragram_300_sl999.txt')
add_lower_and_upper(embeddings_index2, word_index)
add_symbols(embeddings_index2, word_index)
embedding_matrix1 = build_embedding_matrix(word_index,embeddings_index2, max_features, maxlen, lower = True)
del embeddings_index2
gc.collect()

 52%|█████▏    | 107854/207626 [00:00<00:00, 552595.60it/s]

100%|██████████| 207626/207626 [00:00<00:00, 1107349.93it/s]
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  


Added 63871 lower word embeddings
Added 3 upper word embeddings
Added 0 symbol embeddings


0

In [37]:
def build_lstm_gru_model(embedding_matrix, name = ""):
    
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
    x = SpatialDropout1D(.4, seed = seed_nb)(x)
    x = Bidirectional(CuDNNLSTM(128, return_sequences=True,kernel_initializer=glorot_uniform(seed=seed_nb)))(x)
    x = Dropout(.2, seed = seed_nb)(x)
    x = Bidirectional(CuDNNGRU(64, return_sequences=True,kernel_initializer=glorot_uniform(seed=seed_nb)))(x)

    last = Flatten()(Lambda(lambda x: x[:,-1:,:]) (x))
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool,max_pool,last])
    x = Dropout(.3, seed = seed_nb)(x)
    outp = Dense(1, activation="sigmoid",kernel_initializer=he_uniform(seed=seed_nb))(conc)
    
    model = Model(inputs=[inp], outputs=outp, name = name)
    return model

In [38]:
K.clear_session()
model_name = 'lstm_gru'
batch_size = 2048
epochs = [0,13,3] # checked locally

model = build_lstm_gru_model(embedding_matrix1)
lstm_gru_hist = train_model(model,train_X, train_y, epochs,batch_size)
preds_test_all[:,3] = model.predict(test_X, batch_size=batch_size,verbose = True)[:,0]

Epoch 1/13
391837/391837 [==============================] - 68s 174us/step - loss: 0.3993 - acc: 0.8315 - f1: 0.4152
Epoch 2/13
391837/391837 [==============================] - 67s 171us/step - loss: 0.3478 - acc: 0.8527 - f1: 0.5678
Epoch 3/13
391837/391837 [==============================] - 67s 171us/step - loss: 0.3317 - acc: 0.8598 - f1: 0.6019
Epoch 4/13
391837/391837 [==============================] - 67s 171us/step - loss: 0.3217 - acc: 0.8643 - f1: 0.6207
Epoch 5/13
391837/391837 [==============================] - 67s 171us/step - loss: 0.3148 - acc: 0.8667 - f1: 0.6320
Epoch 6/13
391837/391837 [==============================] - 67s 171us/step - loss: 0.3080 - acc: 0.8701 - f1: 0.6436
Epoch 7/13
391837/391837 [==============================] - 67s 171us/step - loss: 0.3020 - acc: 0.8727 - f1: 0.6540
Epoch 8/13
391837/391837 [==============================] - 67s 171us/step - loss: 0.2967 - acc: 0.8749 - f1: 0.6620
Epoch 9/13
391837/391837 [==============================] - 67s 

In [39]:
del model, lstm_gru_hist
gc.collect()

160

In [40]:
del embedding_matrix1
gc.collect()


0

In [41]:
embeddings_index3 = load_word2vec(INPUT_PATH + 'embeddings/GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin')
add_lower_and_upper(embeddings_index3, word_index)
add_numbers(embeddings_index3, word_index)
add_symbols(embeddings_index3, word_index)
embedding_matrix1 = build_embedding_matrix(word_index,embeddings_index3, max_features, maxlen, lower = False)
del embeddings_index3
gc.collect()

  7%|▋         | 14745/207626 [00:00<00:01, 147448.73it/s]

Added 1516 lower word embeddings
Added 6329 upper word embeddings


 46%|████▌     | 95030/207626 [00:00<00:00, 950192.17it/s]

Added 0 number embeddings


100%|██████████| 207626/207626 [00:00<00:00, 982454.36it/s]
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  


Added 0 symbol embeddings


/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:107: RuntimeWarning: overflow encountered in reduce
  arrmean = umr_sum(arr, axis, dtype, keepdims=True)


0

In [42]:
K.clear_session()
model_name = 'parallel_lstm_gru3'
batch_size = 2048
epochs = [0,13,3] # checked locally

model = build_parallel_lstm_gru(embedding_matrix1)
parallel_lstm_gru3_hist = train_model(model,train_X, train_y, epochs,batch_size)
preds_test_all[:,4] = model.predict(test_X, batch_size=batch_size,verbose = True)[:,0]

Epoch 1/13
391837/391837 [==============================] - 63s 160us/step - loss: 0.4080 - acc: 0.8261 - f1: 0.3914
Epoch 2/13
391837/391837 [==============================] - 62s 157us/step - loss: 0.3604 - acc: 0.8468 - f1: 0.5440
Epoch 3/13
391837/391837 [==============================] - 62s 157us/step - loss: 0.3438 - acc: 0.8549 - f1: 0.5819
Epoch 4/13
391837/391837 [==============================] - 62s 157us/step - loss: 0.3345 - acc: 0.8591 - f1: 0.6016
Epoch 5/13
391837/391837 [==============================] - 62s 157us/step - loss: 0.3280 - acc: 0.8619 - f1: 0.6135
Epoch 6/13
391837/391837 [==============================] - 62s 157us/step - loss: 0.3216 - acc: 0.8645 - f1: 0.6243
Epoch 7/13
391837/391837 [==============================] - 62s 157us/step - loss: 0.3169 - acc: 0.8658 - f1: 0.6302
Epoch 8/13
391837/391837 [==============================] - 62s 157us/step - loss: 0.3128 - acc: 0.8684 - f1: 0.6396
Epoch 9/13
391837/391837 [==============================] - 62s 

In [43]:
del model, parallel_lstm_gru3_hist
gc.collect()

164

In [44]:
del embedding_matrix1
gc.collect()

0

These are the weights that were found offline using hillclimbing. These are not really all that precise. At the end we ran out of time so I simply ran things once and this is what was popped out. 

In [45]:
w = np.array([41, 95, 66, 68, 87])

In [46]:
preds = np.zeros((test_X.shape[0]))
for i in range(5):
    preds += w[i]*preds_test_all[:,i]
preds/= np.sum(w)
preds.mean()

0.06502898592737692

Our threshold was set to .34 because this seemed to work reasonably well in our CV. One thing to note is we totally dropped any sort of validation come rollout time in order to maximize the training data our model was exposed to. This ended up having a significant effect on our performance. 

The threshold likely could have been more precisely selected. 

It's unfortunate we weren't able to further exploit the tricks we came across, but glad the decision to give a shot to the time consuming POS tagging step paid off. Once again it shows how important preprocessing is. 

In [47]:
y_te = np.array(preds>0.34, dtype=np.int8)
submit_df = pd.DataFrame({"qid": test_df["qid"], "prediction": y_te})
submit_df.to_csv("submission.csv", index=False)

In [48]:
# submit_df = pd.DataFrame({"qid": test_df["qid"], "prediction": pred})
# submit_df.to_csv("submission.csv", index=False)